In [24]:
import pandas as pd
from openpyxl import  load_workbook
from openpyxl.styles import  Font,PatternFill

file_list = ["2022_年間売上表.xlsx","2023_年間売上表.xlsx"]

df = pd.DataFrame()

for file_name in file_list:
  temp_df = pd.read_excel(file_name)

  df = pd.concat([df,temp_df],ignore_index=True)

df = df[df["売上年"] != "売上年"]

df["金額（千円）"] = df["金額（千円）"].apply(pd.to_numeric,errors="coerce")
df.dropna(subset=["金額（千円）"],inplace=True)
df["金額（千円）"] = df["金額（千円）"].astype(int)

reesult = df.groupby(["商品","売上年"])["金額（千円）"].sum().reset_index()

file_name = "売上集計表.xlsx"

reesult.to_excel(file_name,sheet_name="sheet1",index=False)

wb = load_workbook(file_name)
ws = wb["sheet1"]

column_widths = {
    "A":20,
    "B":15,
    "C":15
}

for col,width in column_widths.items():
  ws.column_dimensions[col].width = width

header_fill = PatternFill(start_color="F2F2F2",end_color="F2F2F2",fill_type="solid")

for cell in ws[1]:
    cell.fill = header_fill

bold_font = Font(bold=True,size=14)

for row in ws.iter_rows(min_row=1,max_row=ws.max_row,min_col=1,max_col=ws.max_column):
  for cell in row:
      cell.font = bold_font

reesult.to_excel("売上集計表.xlsx",sheet_name="sheet1",index=False)

wb.save(file_name)